In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

stock_market_prediction_and_sentimental_analysis_path = kagglehub.competition_download('stock-market-prediction-and-sentimental-analysis')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stock-market-prediction-and-sentimental-analysis/sample_submission.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Combined_News_DJIA(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_DJIA_Table.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/TEST_Redit_news.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/RedditNews(train).csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/Test_Combined_News.csv
/kaggle/input/stock-market-prediction-and-sentimental-analysis/DJIA_table(train).csv


In [ ]:
!pip install ta vaderSentiment

In [ ]:
!pip install ta vaderSentiment
import nltk
nltk.download('vader_lexicon')

import numpy as np
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import ta
from sklearn.linear_model import Ridge
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Load Data
base_path = "/kaggle/input/stock-market-prediction-and-sentimental-analysis/"
cntr = pd.read_csv(base_path + "Combined_News_DJIA(train).csv")
djtr = pd.read_csv(base_path + "DJIA_table(train).csv")
cnts = pd.read_csv(base_path + "Test_Combined_News.csv")
djts = pd.read_csv(base_path + "Test_DJIA_Table.csv")

# Fix dates formatting securely
djtr["Date"] = pd.to_datetime(djtr["Date"], format="mixed", dayfirst=True, errors='coerce')
djts["Date"] = pd.to_datetime(djts["Date"], format="mixed", dayfirst=True, errors='coerce')
cntr["Date"] = pd.to_datetime(cntr["Date"], format="mixed", dayfirst=True, errors='coerce')
cnts["Date"] = pd.to_datetime(cnts["Date"], format="mixed", dayfirst=True, errors='coerce')

In [ ]:
# Calculate sentiment for EACH headline individually
top_cols = [f"Top{i}" for i in range(1, 26)]

print("Calculating detailed VADER sentiments... (This may take a minute)")
for col in top_cols:
    # Train set
    cntr[f"{col}_sent"] = cntr[col].astype(str).apply(lambda x: sia.polarity_scores(x)['compound'])
    # Test set
    cnts[f"{col}_sent"] = cnts[col].astype(str).apply(lambda x: sia.polarity_scores(x)['compound'])

# Create Aggregate Sentiment Features
sent_cols = [f"Top{i}_sent" for i in range(1, 26)]

# For Train
cntr['sent_mean'] = cntr[sent_cols].mean(axis=1)
cntr['sent_min'] = cntr[sent_cols].min(axis=1) # Captures the most negative news of the day
cntr['sent_max'] = cntr[sent_cols].max(axis=1) # Captures the most positive news of the day

# For Test
cnts['sent_mean'] = cnts[sent_cols].mean(axis=1)
cnts['sent_min'] = cnts[sent_cols].min(axis=1)
cnts['sent_max'] = cnts[sent_cols].max(axis=1)

# Keep only necessary columns for merge
cntr_sent = cntr[['Date', 'sent_mean', 'sent_min', 'sent_max']]
cnts_sent = cnts[['Date', 'sent_mean', 'sent_min', 'sent_max']]

Calculating detailed VADER sentiments... (This may take a minute)


In [ ]:
# 0. Create a flag to separate Train and Test data later
djtr['is_test'] = 0
djts['is_test'] = 1
djts['Close'] = np.nan # Test set target hide kar rahe hain

# Combine temporarily for calculating rolling technical indicators correctly
dj_all = pd.concat([djtr, djts]).sort_values('Date').reset_index(drop=True)

# 1. Shift basic price features
dj_all['prev_close'] = dj_all['Close'].shift(1)
dj_all['prev_open']  = dj_all['Open'].shift(1)
dj_all['prev_high']  = dj_all['High'].shift(1)
dj_all['prev_low']   = dj_all['Low'].shift(1)
dj_all['prev_volume']= dj_all['Volume'].shift(1)

# 2. Add Quantitative Finance Features
# Daily return from yesterday
dj_all['return_1d'] = dj_all['prev_close'].pct_change(1)
# 3-day momentum
dj_all['return_3d'] = dj_all['prev_close'].pct_change(3)

# Volatility (Risk) - 7 day rolling standard deviation of returns
dj_all['volatility_7d'] = dj_all['return_1d'].rolling(7).std()

# Technical Indicators
dj_all['RSI_14'] = ta.momentum.RSIIndicator(close=dj_all['prev_close'], window=14).rsi()
dj_all['MACD']   = ta.trend.MACD(close=dj_all['prev_close']).macd()
dj_all['BB_width'] = ta.volatility.BollingerBands(close=dj_all['prev_close']).bollinger_wband()

# Fill NaNs generated by rolling/shifting (bfill works fine for first few rows)
dj_all.fillna(method='bfill', inplace=True)

# Split back using the is_test flag
djtr_feat = dj_all[dj_all['is_test'] == 0].copy()
djts_feat = dj_all[dj_all['is_test'] == 1].copy()

# Merge with NEW Sentiment features
train_df = pd.merge(djtr_feat, cntr_sent, on='Date', how='inner')
test_df  = pd.merge(djts_feat, cnts_sent, on='Date', how='inner')

# Target (Predicting Return relative to Open)
train_df["Target_Return"] = (train_df["Close"] - train_df["Open"]) / train_df["Open"]
train_df["Target_Label"] = np.where(train_df["Target_Return"] >= 0, 1, 0)

print("Features calculated and data successfully split back into train and test!")

Features calculated and data successfully split back into train and test!


In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = ['prev_close', 'prev_volume', 'return_1d', 'return_3d', 'volatility_7d',
            'RSI_14', 'MACD', 'BB_width', 'sent_mean', 'sent_min', 'sent_max']

X = train_df[features]
y_reg = train_df['Target_Return']
y_clf = train_df['Target_Label']

split = int(len(train_df) * 0.8)
X_train, X_val = X.iloc[:split], X.iloc[split:]
y_train_reg, y_val_reg = y_reg.iloc[:split], y_reg.iloc[split:]
y_train_clf, y_val_clf = y_clf.iloc[:split], y_clf.iloc[split:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 1. Classification (Using Random Forest with strict depth limits)
clf = RandomForestClassifier(n_estimators=200, max_depth=4, min_samples_leaf=5, random_state=42)
clf.fit(X_train_scaled, y_train_clf)

val_preds_clf = clf.predict(X_val_scaled)
print("--- DIRECTIONAL PREDICTION (Random Forest) ---")
print("Accuracy:", accuracy_score(y_val_clf, val_preds_clf))
print(classification_report(y_val_clf, val_preds_clf))

# 2. Regression (Stronger Ridge to ignore noise)
reg = Ridge(alpha=50.0)
reg.fit(X_train_scaled, y_train_reg)

val_preds_reg = reg.predict(X_val_scaled)
print("\n--- RETURN PREDICTION (Ridge) ---")
print("RMSE:", np.sqrt(mean_squared_error(y_val_reg, val_preds_reg)))
print("R2 Score:", r2_score(y_val_reg, val_preds_reg))

--- DIRECTIONAL PREDICTION (Random Forest) ---
Accuracy: 0.4718498659517426
              precision    recall  f1-score   support

           0       0.40      0.13      0.20       185
           1       0.49      0.81      0.61       188

    accuracy                           0.47       373
   macro avg       0.44      0.47      0.40       373
weighted avg       0.44      0.47      0.40       373


--- RETURN PREDICTION (Ridge) ---
RMSE: 0.008845547373688697
R2 Score: -0.0007924196556279295


In [ ]:
# 1. Fill NaN values in features to prevent the ValueError
X_test = test_df[features].copy()
X_test = X_test.fillna(method='ffill').fillna(method='bfill').fillna(0)

# 2. Scale the clean data
X_test_scaled = scaler.transform(X_test)

# 3. Predict Test Returns
test_predicted_returns = reg.predict(X_test_scaled)

# 4. Fill NaNs in 'Open' price just in case, before math operation
test_df['Open'] = test_df['Open'].fillna(method='ffill').fillna(method='bfill')

# Calculate Predicted Close Price using actual Test Open price
test_df['Predicted_Close'] = test_df['Open'] * (1 + test_predicted_returns)

# Format for Kaggle Submission (Descending order by Date)
submission = test_df[['Date', 'Predicted_Close']].copy()
submission = submission.sort_values("Date", ascending=False).reset_index(drop=True)

# Rename columns and add ID
submission.rename(columns={"Predicted_Close": "Close"}, inplace=True)
submission.drop(columns=['Date'], inplace=True)
submission.insert(0, "id", range(1, len(submission) + 1))

# Save
submission.to_csv("submission.csv", index=False)
print("Successfully generated valid submission.csv!")
print(submission.head())

Successfully generated valid submission.csv!
   id         Close
0   1  17946.675813
1   2  17739.479633
2   3  17483.737463
3   4  17221.063389
4   5  17384.447244
